<a href="https://colab.research.google.com/github/SCS-Technology-and-Innovation/IntroComp/blob/main/random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pseudorandomness

Nothing a computer can do is ever truly *random*. Through clever use of mathematics (calculations) and physics (readings from sensors such as temperature), it is possible to give the impression of randomness. With more work, even statistically convincing randomness is viable to some degree.

Most programming languages contain some routines for (pseudo-)random number generation within their built-in or default packages, whereas the finer ones (with hope of statistical validity, although not necessarily cryptographic strength) tend to be included within additional optional libraries that are installed as needed.

The basic form is usually the generation of uniformly distributed values within the interval $[0, 1]$.

In [1]:
from random import random

print(random())

0.6703847342614631


If we want a `True` or a `False`, we can threshold these at the midpoint.

In [19]:
n = 10
print([ random() < 0.5 for k in range(n) ])

[True, False, True, False, False, False, False, False, True, False]


If we obtain $n$ of these, we would expect $n/2$ to be below 0.5.

In [3]:
n = 10000
print(sum( [random() < 0.5 for k in range(n) ]))

4999


If we repeat the experiment (repetitions are called replicas), we can see how this varies more for low values of $n$.

In [4]:
r = 10
n = 1000
for replica in range(r):
  print(sum( [random() < 0.5 for k in range(n) ]))

474
528
517
510
486
518
494
502
504
513


What if we need a value in an interval $[a, b]$ instead of $[0, 1]$? We can either do the math ourselves or rely on a library routine, whenever one is provided.

In [9]:
from random import uniform

r = 10
a = -10
b = 25

print([ uniform(a, b) for replica in range(r) ])


def myown(low, high):
  span = high - low
  return low + span * random()


print([ myown(a, b) for replica in range(r) ])

[-2.218322719146329, 17.869916634976047, -9.10079325918374, 16.149618268123728, 5.935449050513652, 17.176356592397394, 21.66030543342165, 11.024252244109096, 11.20976353013059, 21.72934598439601]
[-6.4150277159307265, 3.6065938652220613, 8.283623349402081, 22.325241962746325, -4.39051697185317, 12.93261613097292, 0.6380701789965713, 16.199295745626642, -0.7401003624776266, -5.718131711739346]


Should we need these to be *integers*, we can `round` or look for another routine provided by a library.

In [14]:
from random import randrange
from math import floor

r = 10
a = -10
b = 25

def myownint(low, high):
  return round(myown(low, high))

print([ myownint(a, b) for replica in range(r) ])
print([ randrange(a, b + 1) for replica in range(r) ])

[23, 23, 16, 6, 17, 4, 22, 2, -1, 24]
[14, 6, 14, 4, 8, 22, 9, 23, 17, 25]


Note that `randrange` is like `range` and hence the end point $b$ is not included as a possible value so we have to add one to get $b$s as well.

To pick an element from a list (or an array), one must generate an index to indicate which one is chosen. This boils down to generating integers between zero (inclusive) and the length of the list (or the array; exclusive). There may be ready-made routines as well for this purpose.

In [29]:
data = [ 5, 19, 67, 24 ]

from random import choice

def ownpick(l):
  return l[myownint(0, len(l) - 1)]


for replica in range(r):
  print(choice(data), ownpick(data))

67 67
5 5
19 67
67 5
67 24
5 67
67 24
19 19
24 67
67 19


We might not want each element to be equally likely when picking, but instead give a higher likelihood to some and a lower one to others (this is called *roulette-wheel* selection).

In [37]:
weights = [ 6, 1, 5, 3 ] # any non-negative weights that indicate the relative likelihoods of the elements
assert len(weights) == len(data) # need as many as there are options to choose from

from random import choices

print(choices(data, weights, k = 2)) # pick any two, using these weights

[24, 19]


Writing code to pick one element in this fashion ourselves is not hard. We need to generate a value between zero and the sum of the weights, then accumulate weights until we reach the generated value, and the index at which this happens is chosen.

In [44]:
def pickweight(l, w):
  high = sum(w)
  v = myown(0, high)
  accumulated = 0
  max = len(l)
  i = 0
  while accumulated < v:
    accumulated += w[i]
    if i < max - 1:
      i += 1
  return l[i]

for replica in range(r):
  print(pickweight(data, weights))

24
19
24
19
19
19
19
24
19
19


To pick several would amount to repeating this until the desired quantity is obtained. If we want distint elements, we can store them in a `set` so that we do not get repetitions.

In [45]:
goal = 2
gotten = set()
while len(gotten) < goal:
  gotten.add(pickweight(data, weights))

print(gotten)

{24, 67}


*Shuffling* a list is a common task, achieved technically picking elements from it at random until everything has been picked. It is not a lot of work to write such a routine oneself, but often it is provided in a library.

In [34]:
from random import shuffle

shuffle(data)
print(data)

[5, 19, 67, 24]
